In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold

from sklearn.cross_validation import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso, Ridge

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
def normalize_data(X):
    return pd.DataFrame(
        StandardScaler().fit_transform(X),
        index = X.index, 
        columns=X.columns
    )

def logEstimation(X, y):
    grid = {'C': np.power(10.0, np.arange(-5, 6))}
    kf=KFold(y.size, n_folds=5, shuffle=True, random_state=241)
    clf=LogisticRegression(random_state=241)
    gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=kf)
    gs.fit(X, y)
    return gs

In [4]:
train_matches = pd.read_csv('data/train.csv')
test_matches = pd.read_csv('data/test.csv')

In [5]:
gold = pd.read_csv('data/gold.csv', index_col='mid')
gold = gold[gold.times == 600]
gold.drop('times', 1, inplace=True)

norm_gold = pd.DataFrame(index=gold.index)
for col in gold.columns:
    norm_gold['gold_' + col] = gold[col]
norm_gold.head()

,gold_player_0,gold_player_1,gold_player_2,gold_player_3,gold_player_4,gold_player_5,gold_player_6,gold_player_7,gold_player_8,gold_player_9
mid,,,,,,,,,,
0,3454,5206,2613,4426,5755,4072,3997,5917,1725,6384
1,2477,5760,3816,4353,5759,7659,5066,2748,4440,4623
2,3604,1948,8581,4390,2869,3096,2301,5130,2530,2491
3,3457,5464,4432,2961,4314,3345,4791,1906,5328,2247
4,3675,4103,5154,3030,2076,3920,3494,3392,4458,2220


In [6]:
radiant_gold = gold[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].sum(axis=1)
dire_gold = gold[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].sum(axis=1)
radiant_carry_gold = gold[['player_0', 'player_1', 'player_2', 'player_3', 'player_4']].max(axis=1)
dire_carry_gold = gold[['player_5', 'player_6', 'player_7', 'player_8', 'player_9']].max(axis=1)
norm_gold = normalize_data(norm_gold)
norm_gold.reset_index(inplace=True)
norm_gold.head()

,mid,gold_player_0,gold_player_1,gold_player_2,gold_player_3,gold_player_4,gold_player_5,gold_player_6,gold_player_7,gold_player_8,gold_player_9
0,0,-0.460819,0.883620,-1.022316,0.302363,1.318719,0.005361,0.009184,1.420892,-1.704481,1.827721
1,1,-1.176482,1.291373,-0.146754,0.248927,1.321671,2.659065,0.808769,-0.933944,0.328328,0.498696
2,2,-0.350943,-1.514322,3.321283,0.276011,-0.811136,-0.716695,-1.259380,0.836084,-1.101751,-1.110322
3,3,-0.458622,1.073512,0.301579,-0.770018,0.255267,-0.532482,0.603076,-1.559622,0.993203,-1.294469
4,4,-0.298934,0.071793,0.827062,-0.719510,-1.396367,-0.107090,-0.367047,-0.455398,0.341805,-1.314846


In [7]:
train = pd.merge(train_matches, norm_gold, on='mid', how='left')
test = pd.merge(test_matches, norm_gold, on='mid', how='left')
X_train = train.drop(['radiant_won', 'mid'], 1)
y_train = train.radiant_won

# Usual_gold_score

In [65]:
new_gold = pd.read_csv('data/gold.csv', index_col='mid')
new_gold = new_gold[new_gold.times == 600]
new_gold.drop('times', 1, inplace=True)
heroes = pd.read_csv('data/heroes.csv', index_col='mid')

mean_gold = np.zeros(111)
for hero_num in range(111):
    hero_values = []
    for player_num in range(10):
        colname = 'player_{}'.format(player_num)
        hero_index = heroes.loc[heroes[colname] == hero_num].index
        hero_values.extend(new_gold.loc[hero_index][colname].values)
    mean_gold[hero_num] = int(np.mean(hero_values))

mean_gold = new_mean_gold
mean_gold[0] = 100000

mean_predict_gold = heroes.apply(lambda x: mean_gold[x])

temp_gold_score = new_gold / mean_predict_gold
usual_gold_score = pd.DataFrame(index=temp_gold_score.index)
for col in temp_gold_score.columns:
    usual_gold_score['usual_gold_score_' + col] = temp_gold_score[col]
usual_gold_score.reset_index(level=0, inplace=True)

radiant_rank_gold_score = pd.DataFrame(data = np.sort(usual_gold_score.values[:,1:6]),
                                       index=adapt_gold_score.index,
                                       columns = ['gold_score_rank_{0}'.format(i) for i in range(5)]) 

dire_rank_gold_score = pd.DataFrame(data = np.sort(usual_gold_score.values[:,6:11]),
                                       index=adapt_gold_score.index,
                                       columns = ['gold_score_rank_{0}'.format(i) for i in range(5)]) 

gold_rank_dif = pd.DataFrame(data = np.sort(gold.values[:,0:5]) - np.sort(gold.values[:,5:10]),
                             index=gold.index,
                             columns = ['gold_rank_{}'.format(i) for i in range(5)]) 

gold_stats_usual = pd.DataFrame(index=gold.index)
gold_stats_usual['gold_dif'] = radiant_gold - dire_gold
gold_stats_usual['top_gold_dif'] = gold_rank_dif.gold_rank_4
gold_stats_usual['second_gold_dif'] = gold_rank_dif.gold_rank_3
# gold_stats_usual['third_gold_dif'] = gold_rank_dif.gold_rank_2
gold_stats_usual['sum_usual_gold_score_dif'] = radiant_rank_gold_score.sum(1) - dire_rank_gold_score.sum(1)
gold_stats_usual['top_usual_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_4 - dire_rank_gold_score.gold_score_rank_4
gold_stats_usual['second_usual_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_3 - dire_rank_gold_score.gold_score_rank_3
gold_stats_usual['third_usual_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_2 - dire_rank_gold_score.gold_score_rank_2
gold_stats_usual = normalize_data(gold_stats_usual)
gold_stats_usual.reset_index(inplace=True)
gold_stats_usual.to_csv('processing_tables/gold_stats_usual_dif.csv', index=None)
gold_stats_usual.head()



,mid,gold_dif,top_gold_dif,second_gold_dif,sum_usual_gold_score_dif,top_usual_gold_score_dif,second_usual_gold_score_dif,third_usual_gold_score_dif
0,0,-0.191463,-0.463171,-0.665978,0.859705,0.548728,0.619844,1.010669
1,1,-0.657847,-1.316314,0.611509,-0.047924,0.125667,0.121465,0.338286
2,2,1.556804,2.277633,1.158354,1.242823,2.660495,0.305003,0.065889
3,3,0.793066,0.050730,-0.345696,0.821488,0.842257,0.342623,0.234499
4,4,0.130693,0.426918,0.147465,-0.725224,0.138679,-0.569823,-1.043559


# adaptive gold_score

In [10]:
new_gold = pd.read_csv('data/gold.csv', index_col='mid')
new_gold = new_gold[new_gold.times == 600]
new_gold.drop('times', 1, inplace=True)
heroes = pd.read_csv('data/heroes.csv', index_col='mid')

mean_gold = np.zeros(111)
for hero_num in range(111):
    hero_values = []
    for player_num in range(10):
        colname = 'player_{}'.format(player_num)
        hero_index = heroes.loc[heroes[colname] == hero_num].index
        hero_values.extend(new_gold.loc[hero_index][colname].values)
    mean_gold[hero_num] = int(np.mean(hero_values))

print 'Source mean gold:\n', mean_gold
print 'Sorted mean gold:\n', np.sort(mean_gold)
new_mean_gold = np.array(map(lambda x: x if x > 4500 else 4500 + 5*(4500 - x), mean_gold))
# new_mean_gold = np.array(map(lambda x: x if x > 4500 else 4500 + int((4500 - x)*2 / 1000), mean_gold))
mean_gold = new_mean_gold
mean_gold[0] = 100000
print 'After threshold:\n', mean_gold

Source mean gold:
[ 1686.  4793.  4149.  3651.  3124.  3789.  3245.  3224.  4364.  4589.
  3101.  4808.  3217.  4635.  4535.  5610.  3314.  4150.  3062.  2610.
  3785.  3000.  4603.  3873.  4939.  4383.  3343.  4516.  4352.  4605.
  4443.  4447.  3635.  3305.  5088.  4666.  4527.  4603.  4285.  2838.
  2793.  3345.  3678.  3807.  4264.  4556.  4426.  4225.  4485.  4493.
  2736.  4022.  4648.  4187.  4117.  4461.  4843.  3359.  4398.  4403.
  2970.  2825.  3845.  3786.  4562.  4364.  2684.  4646.  4162.  2895.
  4601.  3486.  4865.  4075.  6446.  4589.  3293.  4323.  4425.  4224.
  2965.  4139.  4968.  4354.  4374.  4719.  2680.  2754.  2921.  3712.
  2940.  2838.  2795.  3290.  3938.  3838.  3374.  3016.  2959.  3370.
  3129.  4483.  4430.  2908.  2873.  4453.  3710.  4110.  3927.  2862.
  4122.]
Sorted mean gold:
[ 1686.  2610.  2680.  2684.  2736.  2754.  2793.  2795.  2825.  2838.
  2838.  2862.  2873.  2895.  2908.  2921.  2940.  2959.  2965.  2970.
  3000.  3016.  3062.  3101.  31

In [67]:
mean_predict_gold = heroes.apply(lambda x: mean_gold[x])
# mean_predict_gold.head()

temp_gold_score = new_gold / mean_predict_gold
adapt_gold_score = pd.DataFrame(index=temp_gold_score.index)
for col in temp_gold_score.columns:
    adapt_gold_score['adapt_gold_score_' + col] = temp_gold_score[col]
adapt_gold_score.reset_index(level=0, inplace=True)

radiant_rank_gold_score = pd.DataFrame(data = np.sort(adapt_gold_score.values[:,1:6]),
                                       index=adapt_gold_score.index,
                                       columns = ['gold_score_rank_{0}'.format(i) for i in range(5)]) 
# radiant_rank_gold_score = normalize_data(radiant_rank_gold_score)
dire_rank_gold_score = pd.DataFrame(data = np.sort(adapt_gold_score.values[:,6:11]),
                                       index=adapt_gold_score.index,
                                       columns = ['gold_score_rank_{0}'.format(i) for i in range(5)]) 
# dire_rank_gold_score = normalize_data(dire_rank_gold_score)
dire_rank_gold_score.head()
# print 

,gold_score_rank_0,gold_score_rank_1,gold_score_rank_2,gold_score_rank_3,gold_score_rank_4
0,0.137725,0.370951,0.846922,0.990382,1.162932
1,0.214520,0.818417,0.857143,0.961522,1.188179
2,0.197151,0.211489,0.350822,0.553005,1.054471
3,0.156166,0.175410,0.267066,1.041295,1.078761
4,0.163235,0.395921,0.448256,0.722471,0.977203


In [69]:
gold_rank_dif = pd.DataFrame(data = np.sort(gold.values[:,0:5]) - np.sort(gold.values[:,5:10]),
                             index=gold.index,
                             columns = ['gold_rank_{}'.format(i) for i in range(5)]) 

gold_stats_adapt = pd.DataFrame(index=gold.index)
gold_stats_adapt['gold_dif'] = radiant_gold - dire_gold
gold_stats_adapt['top_gold_dif'] = gold_rank_dif.gold_rank_4
gold_stats_adapt['second_gold_dif'] = gold_rank_dif.gold_rank_3
# gold_stats_adapt['third_gold_dif'] = gold_rank_dif.gold_rank_2
gold_stats_adapt['adapt_score_sum_dif'] = radiant_rank_gold_score.sum(1) - dire_rank_gold_score.sum(1)
gold_stats_adapt['top_adapt_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_4 - dire_rank_gold_score.gold_score_rank_4
gold_stats_adapt['second_adapt_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_3 - dire_rank_gold_score.gold_score_rank_3
gold_stats_adapt['third_adapt_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_2 - dire_rank_gold_score.gold_score_rank_2
gold_stats_adapt = normalize_data(gold_stats_adapt)
gold_stats_adapt.reset_index(inplace=True)
gold_stats_adapt.to_csv('processing_tables/gold_stats_adapt_dif.csv', index=None)
gold_stats_adapt.head()

,mid,gold_dif,top_gold_dif,second_gold_dif,adapt_score_sum_dif,top_adapt_gold_score_dif,second_adapt_gold_score_dif,third_adapt_gold_score_dif
0,0,-0.191463,-0.463171,-0.665978,-0.690054,-0.202450,-0.752267,-0.850836
1,1,-0.657847,-1.316314,0.611509,-0.109702,0.227309,0.932836,0.090742
2,2,1.556804,2.277633,1.158354,1.747569,2.517466,1.418848,0.803090
3,3,0.793066,0.050730,-0.345696,0.138993,-0.407567,-1.315278,1.146762
4,4,0.130693,0.426918,0.147465,0.995747,0.471640,0.017396,0.974948


# threshold gold score

In [52]:
new_gold = pd.read_csv('data/gold.csv', index_col='mid')
new_gold = new_gold[new_gold.times == 600]
new_gold.drop('times', 1, inplace=True)
heroes = pd.read_csv('data/heroes.csv', index_col='mid')

mean_gold = np.zeros(111)
for hero_num in range(111):
    hero_values = []
    for player_num in range(10):
        colname = 'player_{}'.format(player_num)
        hero_index = heroes.loc[heroes[colname] == hero_num].index
        hero_values.extend(new_gold.loc[hero_index][colname].values)
    mean_gold[hero_num] = int(np.mean(hero_values))

mean_gold[0] = 100000
print 'Source:\n', mean_gold
print 'Sorted:\n', np.sort(mean_gold)
new_mean_gold = np.array(map(lambda x: x if x > 4400 else 10000, mean_gold))
mean_gold = new_mean_gold
print 'After threshold:\n', mean_gold

Source:
[ 100000.    4793.    4149.    3651.    3124.    3789.    3245.    3224.
    4364.    4589.    3101.    4808.    3217.    4635.    4535.    5610.
    3314.    4150.    3062.    2610.    3785.    3000.    4603.    3873.
    4939.    4383.    3343.    4516.    4352.    4605.    4443.    4447.
    3635.    3305.    5088.    4666.    4527.    4603.    4285.    2838.
    2793.    3345.    3678.    3807.    4264.    4556.    4426.    4225.
    4485.    4493.    2736.    4022.    4648.    4187.    4117.    4461.
    4843.    3359.    4398.    4403.    2970.    2825.    3845.    3786.
    4562.    4364.    2684.    4646.    4162.    2895.    4601.    3486.
    4865.    4075.    6446.    4589.    3293.    4323.    4425.    4224.
    2965.    4139.    4968.    4354.    4374.    4719.    2680.    2754.
    2921.    3712.    2940.    2838.    2795.    3290.    3938.    3838.
    3374.    3016.    2959.    3370.    3129.    4483.    4430.    2908.
    2873.    4453.    3710.    4110.    392

In [53]:
mean_predict_gold = heroes.apply(lambda x: mean_gold[x])
mean_predict_gold.head()

,player_0,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9
mid,,,,,,,,,,
0,10000.0,10000.0,10000.0,5610.0,10000.0,4808.0,10000.0,5088.0,10000.0,6446.0
1,10000.0,4719.0,10000.0,4939.0,4562.0,6446.0,10000.0,10000.0,10000.0,4808.0
2,10000.0,10000.0,4447.0,4646.0,10000.0,10000.0,10000.0,4865.0,4485.0,10000.0
3,10000.0,10000.0,4483.0,10000.0,10000.0,10000.0,4601.0,10000.0,4939.0,10000.0
4,10000.0,5610.0,4589.0,4605.0,10000.0,10000.0,10000.0,4461.0,4562.0,10000.0


In [57]:
temp_gold_score = new_gold / mean_predict_gold
adapt_gold_score = pd.DataFrame(index=temp_gold_score.index)
for col in temp_gold_score.columns:
    adapt_gold_score['adapt_gold_score_' + col] = temp_gold_score[col]
adapt_gold_score.reset_index(level=0, inplace=True)

radiant_rank_gold_score = pd.DataFrame(data = np.sort(gold_score_by_mean.values[:,1:6]),
                                       index=gold_score_by_mean.index,
                                       columns = ['gold_score_rank_{0}'.format(i) for i in range(5)]) 
# radiant_rank_gold_score = normalize_data(radiant_rank_gold_score)
dire_rank_gold_score = pd.DataFrame(data = np.sort(gold_score_by_mean.values[:,6:11]),
                                       index=gold_score_by_mean.index,
                                       columns = ['gold_score_rank_{0}'.format(i) for i in range(5)]) 
# dire_rank_gold_score = normalize_data(dire_rank_gold_score)
dire_rank_gold_score.head()

,gold_score_rank_0,gold_score_rank_1,gold_score_rank_2,gold_score_rank_3,gold_score_rank_4
0,0.1725,0.3997,0.846922,0.990382,1.162932
1,0.2748,0.4440,0.506600,0.961522,1.188179
2,0.2301,0.2491,0.309600,0.564103,1.054471
3,0.1906,0.2247,0.334500,1.041295,1.078761
4,0.2220,0.3494,0.392000,0.760368,0.977203


In [66]:
gold_rank_dif = pd.DataFrame(data = np.sort(gold.values[:,0:5]) - np.sort(gold.values[:,5:10]),
                             index=gold.index,
                             columns = ['gold_rank_{}'.format(i) for i in range(5)]) 

gold_stats_threshold = pd.DataFrame(index=new_gold.index)
gold_stats_threshold['gold_dif'] = radiant_gold - dire_gold
gold_stats_threshold['top_gold_dif'] = gold_rank_dif.gold_rank_4
gold_stats_threshold['second_gold_dif'] = gold_rank_dif.gold_rank_3
gold_stats_threshold['third_gold_dif'] = gold_rank_dif.gold_rank_2
gold_stats_threshold['threshold_score_sum_dif'] = radiant_rank_gold_score.sum(1) - dire_rank_gold_score.sum(1)
gold_stats_threshold['top_threshold_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_4 - dire_rank_gold_score.gold_score_rank_4
gold_stats_threshold['second_threshold_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_3 - dire_rank_gold_score.gold_score_rank_3
gold_stats_threshold['third_threshold_gold_score_dif'] = radiant_rank_gold_score.gold_score_rank_2 - dire_rank_gold_score.gold_score_rank_2
gold_stats_threshold = normalize_data(gold_stats_threshold)
gold_stats_threshold.reset_index(inplace=True)
gold_stats_threshold.to_csv('processing_tables/gold_stats_threshold_dif.csv', index=None)
gold_stats_threshold.head()

,mid,gold_dif,top_gold_dif,second_gold_dif,third_gold_dif,threshold_score_sum_dif,top_threshold_gold_score_dif,second_threshold_gold_score_dif,third_threshold_gold_score_dif
0,0,-0.191463,-0.463171,-0.665978,0.361294,-1.251202,-1.140037,-1.157989,-1.121482
1,1,-0.657847,-1.316314,0.611509,-0.275577,0.702919,0.197841,0.712788,1.299354
2,2,1.556804,2.277633,1.158354,1.096144,1.497239,2.588696,1.050663,0.180734
3,3,0.793066,0.050730,-0.345696,0.988979,-0.309281,-0.292735,-1.380092,0.339919
4,4,0.130693,0.426918,0.147465,0.184725,0.436196,0.411904,-0.086841,0.923760


In [41]:
gold_stats = pd.DataFrame(index=gold.index)
gold_stats['radiant_gold'] = radiant_gold
gold_stats['dire_gold'] = dire_gold
gold_stats['radiant_top_gold'] = rank_gold.radiant_gold_rank_4
gold_stats['radiant_second_gold'] = rank_gold.radiant_gold_rank_3
gold_stats['dire_top_gold'] = rank_gold.dire_gold_rank_4
gold_stats['dire_second_gold'] = rank_gold.dire_gold_rank_3
# gold_stats = normalize_data(gold_stats)
gold_stats.reset_index(inplace=True)

gold_stats['radiant_top_gold_score'] = radiant_rank_gold_score.radiant_gold_score_rank_4
gold_stats['radiant_second_gold_score'] = radiant_rank_gold_score.radiant_gold_score_rank_3
gold_stats['dire_top_gold_score'] = dire_rank_gold_score.dire_gold_score_rank_4
gold_stats['dire_second_gold_score'] = dire_rank_gold_score.dire_gold_score_rank_3
gold_stats.head()

,mid,radiant_gold,dire_gold,radiant_top_gold,radiant_second_gold,dire_top_gold,dire_second_gold,radiant_top_gold_score,radiant_second_gold_score,dire_top_gold_score,dire_second_gold_score
0,0,21454,22095,5755,5206,6384,5917,-1.337223,-0.958130,0.249037,0.689982
1,1,22165,24536,5760,5759,7659,5066,0.628305,1.584879,0.354146,0.575621
2,2,21392,15548,8581,4390,5130,3096,3.398393,0.498060,-0.202526,-0.999247
3,3,20628,17617,5464,4432,5328,4791,-0.508247,-1.072844,-0.101398,0.891740
4,4,18038,17484,5154,4103,4458,3920,0.050132,-0.343672,-0.524219,-0.221502


In [163]:
train = pd.merge(train_matches, gold_stats, on='mid', how='left')
test = pd.merge(test_matches, gold_stats, on='mid', how='left')
X_train = train.drop(['radiant_won', 'mid'], 1)
y_train = train.radiant_won
X_train = normalize_data(X_train)

In [164]:
clf=LogisticRegression(random_state=241, C=0.1)
print np.mean(cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc'))
clf=Ridge(alpha=0.0001)
print np.mean(cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc'))
clf=Lasso(alpha=0.0001, max_iter=6000)
print np.mean(cross_val_score(clf, X_train, y_train, cv=5, scoring='roc_auc'))

0.709279790104
0.709496303471
0.709498297181


In [165]:
gold_stats.to_csv('processing_tables/gold_stat.csv', index=None)

In [43]:
gold_stats_dif = pd.DataFrame(index=gold.index)
gold_stats_dif['sum_gold_dif'] = radiant_gold - dire_gold
gold_stats_dif['gold_top_dif'] = rank_gold.radiant_gold_rank_4 - rank_gold.dire_gold_rank_4
gold_stats_dif['gold_second_dif'] = rank_gold.radiant_gold_rank_3 - rank_gold.dire_gold_rank_3
gold_stats_dif = normalize_data(gold_stats_dif)

gold_stats_dif['radiant_top_gold_score'] = radiant_rank_gold_score.radiant_gold_score_rank_4
gold_stats_dif['radiant_second_gold_score'] = radiant_rank_gold_score.radiant_gold_score_rank_3
gold_stats_dif['dire_top_gold_score'] = dire_rank_gold_score.dire_gold_score_rank_4
gold_stats_dif['dire_second_gold_score'] = dire_rank_gold_score.dire_gold_score_rank_3

gold_stats_dif.reset_index(inplace=True)
gold_stats_dif.to_csv('processing_tables/gold_stat_dif.csv', index=None)
gold_stats_dif.head()

,mid,sum_gold_dif,gold_top_dif,gold_second_dif,radiant_top_gold_score,radiant_second_gold_score,dire_top_gold_score,dire_second_gold_score
0,0,-0.191463,-0.463171,-0.665978,-1.337223,-0.958130,0.249037,0.689982
1,1,-0.657847,-1.316314,0.611509,0.628305,1.584879,0.354146,0.575621
2,2,1.556804,2.277633,1.158354,3.398393,0.498060,-0.202526,-0.999247
3,3,0.793066,0.050730,-0.345696,-0.508247,-1.072844,-0.101398,0.891740
4,4,0.130693,0.426918,0.147465,0.050132,-0.343672,-0.524219,-0.221502
